# Metaheuristic Optimization for Mixed-Precision Quantization of OPT-1.3B

**Prepared by:** Hermela Wosene, Hiwot Teshome, and Melat Dagnachew  
**Model:** OPT-1.3B  
**Baseline:** AWQ  
**Objective:** Minimize perplexity on WikiText-2 using nature-inspired optimization

---

## Project Scope

This notebook implements nature-inspired optimization algorithms to search for per-layer mixed-bit quantization configurations for the OPT-1.3B model. The goal is to identify quantization settings (3-bit or 4-bit) per transformer layer that outperform the AWQ baseline in perplexity, while maintaining efficient inference constraints.

We use Differential Evolution (DE) as the first search strategy and evaluate each configuration using AWQ’s fake quantization mode. This setup allows rapid, hardware-agnostic performance estimation during optimization.


## Imports and Constants

### 1. Environment Setup and Global Configuration

This section defines essential constants such as model path, number of transformer layers, valid quantization bit widths, and output storage locations.

The model is assumed to be already downloaded and AWQ installed and patched in a prior notebook (`awq_baseline.ipynb`). This notebook only focuses on quantization search and evaluation.


In [1]:
import random
import numpy as np
import subprocess
import os
import time

NUM_LAYERS = 24  # OPT-1.3B
BIT_OPTIONS = [3, 4]  # Allowed bit widths
AWQ_CACHE = "/content/drive/MyDrive/LLM_Quant_Project/awq_cache"
MODEL_PATH = "/content/opt-1.3b"


### 2. Layer-wise Quantization Config Generator

Each candidate solution is a list of 24 integers (either 3 or 4), representing the weight precision used for each transformer layer. This helper function converts such a list into a format compatible with AWQ.

For now, we simulate this by saving a dummy `.pt` file in the format AWQ expects. Later, this can be replaced with actual `run_awq()` outputs if we choose to run real quantization.


## Convert Layer Vector → AWQ File

In [2]:
def save_quant_config(bits, out_path):
    """
    Convert list like [4,3,4,...] to AWQ-style .pt config
    For now, simulate with dummy AWQ-like placeholder file
    """
    import torch
    config = {
        "q_config": {
            "w_bit": bits,
            "q_group_size": [128] * len(bits),
            "zero_point": True
        }
    }
    torch.save(config, out_path)


### 3. Evaluation Function

This function saves a candidate configuration and evaluates it using the AWQ framework. We invoke the AWQ CLI with `--q_backend fake`, which simulates quantized inference for evaluation purposes without requiring real low-bit kernels.

The output perplexity is parsed and returned. If anything fails during evaluation (e.g., timeout or malformed config), a fallback value of infinity is returned to penalize the candidate.


In [3]:
def evaluate_config(bits):
    """
    Save config → run AWQ eval → return perplexity
    """
    config_path = f"{AWQ_CACHE}/temp_config.pt"
    save_quant_config(bits, config_path)

    cmd = [
        "python", "-m", "awq.entry",
        "--model_path", MODEL_PATH,
        "--tasks", "wikitext",
        "--w_bit", "4",
        "--q_group_size", "128",
        "--load_awq", config_path,
        "--q_backend", "fake"
    ]

    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=180)
        output = result.stdout
        for line in output.splitlines():
            if "ppl =" in line:
                ppl = float(line.split("ppl =")[-1].strip())
                return ppl
    except Exception as e:
        print(" Failed to eval:", e)
        return float("inf")


### 4. Metaheuristic Optimization: Differential Evolution (DE)

We implement a simple version of DE with random mutation between population members. Each candidate is evaluated based on perplexity, and the best configuration is preserved across generations.

This algorithm allows exploration of the discrete search space (3/4-bit decisions) without needing gradients or internal model knowledge.


In [4]:
def random_candidate():
    return [random.choice(BIT_OPTIONS) for _ in range(NUM_LAYERS)]

def mutate(target, donor, F=0.5):
    return [random.choice([t, d]) for t, d in zip(target, donor)]

def differential_evolution(pop_size=6, generations=5):
    pop = [random_candidate() for _ in range(pop_size)]
    scores = [evaluate_config(p) for p in pop]

    for g in range(generations):
        print(f"\n Generation {g}")
        for i in range(pop_size):
            x = pop[i]
            a, b = random.sample([p for j, p in enumerate(pop) if j != i], 2)
            trial = mutate(a, b)
            trial_score = evaluate_config(trial)

            if trial_score < scores[i]:
                pop[i], scores[i] = trial, trial_score
                print(f"  ✔ New best for individual {i}: {trial_score:.3f}")

    best_idx = np.argmin(scores)
    return pop[best_idx], scores[best_idx]


# Section 5: Particle Swarm Optimization
# Swarm algorithm using discrete bit flipping in velocity updates

In [7]:
def run_pso(swarm_size=6, iterations=5):
  swarm = [random_candidate() for _ in range(swarm_size)]
  velocities = [random_candidate() for _ in range(swarm_size)]
  personal_best = swarm[:]
  p_scores = [evaluate_config(p) for p in swarm]
  global_best = personal_best[np.argmin(p_scores)]

  for t in range(iterations):
    print(f"\n Iteration {t}")
  for i in range(swarm_size):
    candidate = []
  for j in range(NUM_LAYERS):
    flip = random.random() < 0.5
    candidate.append(global_best[j] if flip else swarm[i][j])
    score = evaluate_config(candidate)
  if score < p_scores[i]:
    personal_best[i], p_scores[i] = candidate, score
    print(f" Particle {i} improved: PPL = {score:.3f}")
    global_best = personal_best[np.argmin(p_scores)]

  return global_best, min(p_scores)




# Section 6: Simulated Annealing
# Physics-inspired stochastic hill-climber with temperature schedule

In [9]:
def run_sa(iters=25, temp=1.0, cooling=0.9):
    current = random_candidate()
    current_score = evaluate_config(current)


    for i in range(iters):
        neighbor = current[:]
        idx = random.randint(0, NUM_LAYERS - 1)
        neighbor[idx] = 3 if current[idx] == 4 else 4
        new_score = evaluate_config(neighbor)


        if new_score < current_score or random.random() < np.exp((current_score - new_score) / temp):
            current, current_score = neighbor, new_score
            print(f" Accepted new config at T={temp:.3f}, PPL={new_score:.3f}")
        temp *= cooling


    return current, current_score

# Section 7: Algorithm Selector + Logging
# Choose which algorithm to run, log best result to CSV

In [11]:
RUN_ALGO = "de" # Change to "pso" or "sa" to run others


if RUN_ALGO == "de":
    # Note: `run_de` function is not defined in the provided notebook state.
    # Assuming `differential_evolution` should be called here based on context.
    # Or `run_de` is a placeholder for a function that will be defined later.
    # For now, I'll use `differential_evolution` as it's defined and imported.
    best_cfg, best_ppl = differential_evolution()
elif RUN_ALGO == "pso":
    best_cfg, best_ppl = run_pso()
elif RUN_ALGO == "sa":
    best_cfg, best_ppl = run_sa()


print("\n🏆 Best PPL:", best_ppl)
print("🧠 Best Config:", best_cfg)

# Note: RESULTS_PATH is not defined in the provided notebook state.
# Assuming `results_path` from the last cell (LY7zPyp5tlx7) should be used.
results_path = "/content/drive/MyDrive/LLM_Quant_Project/results.csv"
import csv # Importing csv module as it's used later

row = ["OPT-1.3B", RUN_ALGO.upper(), "3/4-mixed", "128", "64", best_ppl]
with open(results_path, "a", newline="") as f:
    csv.writer(f).writerow(row)
print(" Result saved to CSV.")

RuntimeError: Parent directory /content/drive/MyDrive/LLM_Quant_Project/awq_cache does not exist.

## Run Opitmization and save best

In [ ]:
best_config, best_score = differential_evolution(pop_size=6, generations=3)
print(" Best perplexity:", best_score)
print(" Best bit config:", best_config)

# Save to results file
import csv

results_path = "/content/drive/MyDrive/LLM_Quant_Project/results.csv"
row = ["OPT-1.3B", "DE", "3/4-mixed", "128", "64", best_score]

with open(results_path, "a", newline="") as f:
    csv.writer(f).writerow(row)
